In [1]:
import warnings
warnings.filterwarnings('ignore')

try:
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [2]:
# path 지정 
train_dir='./cats_and_dogs_small/train'
train_cats_dir=os.path.join(train_dir,'cats')
train_dogs_dir=os.path.join(train_dir,'dogs')

test_dir='./cats_and_dogs_small/test'
test_cats_dir=os.path.join(test_dir,'cats')
test_dogs_dir=os.path.join(test_dir,'dogs')

val_dir='./cats_and_dogs_small/val'
val_cats_dir=os.path.join(val_dir,'cats')
val_dogs_dir=os.path.join(val_dir,'dogs')

In [3]:
# 데이터 전처리 (증대x, 원본 데이터에 약간 변화를 줘서 다양성을 늘림)
train_datagen=ImageDataGenerator(rescale=1./255, # 값을 0~1사이로 바꿈(min-max scaler)
                                shear_range=0.2, # 찌그러트림
                                zoom_range=0.2,
                                horizontal_flip=True) # 수평 방향으로 flip하는가 
# 테스트 데이터에 변화를 줄 필요 없음 (리스케일만 함)
test_datagen=ImageDataGenerator(rescale=1./255)

# flow_from_directory: 디렉토리로부터 파일 가져오기 
train_generator=train_datagen.flow_from_directory(train_dir,
                                                target_size=(150,150),
                                                batch_size=20, # 20개씩 가져옴 
                                                class_mode='binary')

validation_generator=train_datagen.flow_from_directory(val_dir,
                                                target_size=(150,150),
                                                batch_size=20,
                                                class_mode='binary')

test_generator=train_datagen.flow_from_directory(test_dir,
                                                target_size=(150,150),
                                                batch_size=20,
                                                class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [4]:
for data_batch, labels_batch in train_generator:
    print('배치 데이터 크기:', data_batch.shape)
    print('배치 레이블 크기:',labels_batch.shape)
    break

배치 데이터 크기: (20, 150, 150, 3)
배치 레이블 크기: (20,)


In [5]:
model=tf.keras.Sequential([
    tf.keras.layers.Conv2D(input_shape=(150,150,3),kernel_size=(3,3),filters=32),
    tf.keras.layers.MaxPool2D(strides=(2,2)),
    tf.keras.layers.Conv2D(kernel_size=(3,3),filters=64),
    tf.keras.layers.MaxPool2D(strides=(2,2)),
    tf.keras.layers.Conv2D(kernel_size=(3,3),filters=128),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=128,activation='relu'),
    tf.keras.layers.Dropout(rate=0.3),
    tf.keras.layers.Dense(units=1,activation='sigmoid')
])

model.compile(optimizer=tf.keras.optimizers.Adam(), loss='binary_crossentropy',
             metrics=['accuracy'])
model.summary()
history=model.fit(train_generator,steps_per_epoch=100, epochs=5,
                 validation_data=validation_generator, validation_steps=50)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
flatten (Flatten)            (None, 147968)            0         
_________________________________________________________________
dense (Dense)                (None, 128)               1

In [6]:
model.evaluate(test_generator,verbose=0)

[0.6510627269744873, 0.6179999709129333]